In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import datetime

In [31]:
dataF = pd.read_csv("matala2_cosmetics_2019-Nov2.csv")
data = dataF.copy()

c:\users\user\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


---

Q1

In [32]:
data['event_time']= pd.to_datetime(data['event_time'])
data.info()

KeyboardInterrupt: 

In [ ]:
data['DateTime'] = pd.to_datetime(data['event_time'], format='%Y-%m-%d %H:%M:%S.%f')
data['Date'] = pd.to_datetime(data['DateTime'].dt.date)
data['Time'] = pd.to_datetime(data['DateTime'].dt.time, format='%H:%M:%S')
#data.info()
#pd.unique(data['Date'])
data

In [ ]:
data['Time2'] = data.groupby(['user_session'])['Time'].shift(-1).fillna(data['Time'])
data['duration_to_next_event'] = (data['Time2']- data['Time']).dt.seconds
#data.head()

----

Q2

In [ ]:
data['Date2'] = data.groupby(['user_id'])['Date'].shift(1).fillna(data['Date']) # fillna for the first visit

In [ ]:
data['0_1'] = (data['Date'] - data['Date2']).dt.days.apply(lambda x: 1 if x > 5 else 0)

In [ ]:
data['funnel_number']=data.groupby(['user_id'])['0_1'].cumsum()+1
data[data['funnel_number']>1]

----

Q3

In [ ]:
data_copy= data.copy()
data_copy.drop_duplicates(subset='user_session', inplace=True)

In [ ]:
data_copy['index_in_funnel'] = data_copy.groupby(['user_id','funnel_number'])['user_session'].cumcount() + 1
data_copy =data_copy[['user_session','index_in_funnel']]

In [ ]:
data = pd.merge(data, data_copy, on='user_session')

----

Q4

In [ ]:
pd.unique(data['price'])

In [ ]:
def numPrice(x):
    price = re.findall('\S*[0-9]\S*', x)
    res = price[0]
    return float(res)
data['price']=data['price'].apply(numPrice) 


------

Q5

In [ ]:
pd.unique(data['event_type'])

In [ ]:
sns.countplot(x='event_type', data=data)

------

Q6

In [ ]:
session_data=data.copy()

In [ ]:
session_data['number_of_event'] = session_data.groupby(['user_session'])['event_type'].transform('count')
#session_data.head(1)

In [ ]:
dataG=data.groupby('user_session')
dataG_totalTime=dataG['Time'].max()-dataG['Time'].min()
#dataG_totalTime
session_data = pd.merge(session_data, dataG_totalTime, on='user_session')

#session_data

In [ ]:
data_list= session_data.groupby(['user_session','event_type']).agg({
    'product_id': lambda x: list(x)}).reset_index()
pt = pd.pivot_table(data_list, index = 'user_session', columns = 'event_type', values = 'product_id',aggfunc='first').reset_index()
session_data = pd.merge(session_data, pt, on='user_session')
session_data.head(0)

In [ ]:
#session_data.columns.tolist()

In [ ]:
session_data=session_data[['user_id','user_session','funnel_number','number_of_event','Time_y','view','cart','purchase','index_in_funnel']]   

In [ ]:
session_data = session_data.rename(columns={'Time_y': 'Total_tinme'})


In [ ]:
session_data.drop_duplicates(subset='user_session', inplace=True)

In [ ]:
session_data